In [1]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re


In [ ]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

from collections import defaultdict
osm_file = open("dubai_abu-dhabi.osm", "r")




def count_tags(filename):
    tags = {}
    for event, elem in ET.iterparse(filename, events=("start",)):
        t=[]
        if elem.tag:
            t.append(elem.tag)           
            for elem.tag in t:
                if elem.tag in tags.keys():
                    tags[elem.tag] += 1 
                else:
                    tags[elem.tag] = 1
                print tags
count_tags(osm_file)


{'osm': 1}
{'osm': 1, 'bounds': 1}
{'node': 1, 'osm': 1, 'bounds': 1}
{'node': 2, 'osm': 1, 'bounds': 1}
{'node': 3, 'osm': 1, 'bounds': 1}
{'node': 4, 'osm': 1, 'bounds': 1}
{'node': 5, 'osm': 1, 'bounds': 1}
{'node': 6, 'osm': 1, 'bounds': 1}
{'node': 7, 'osm': 1, 'bounds': 1}
{'node': 8, 'osm': 1, 'bounds': 1}
{'node': 9, 'osm': 1, 'bounds': 1}
{'node': 10, 'osm': 1, 'bounds': 1}
{'node': 11, 'osm': 1, 'bounds': 1}
{'node': 12, 'osm': 1, 'bounds': 1}
{'node': 13, 'osm': 1, 'bounds': 1}
{'node': 14, 'osm': 1, 'bounds': 1}
{'node': 15, 'osm': 1, 'bounds': 1}
{'node': 16, 'osm': 1, 'bounds': 1}
{'node': 17, 'osm': 1, 'bounds': 1}
{'node': 18, 'osm': 1, 'bounds': 1}
{'node': 19, 'osm': 1, 'bounds': 1}
{'node': 20, 'osm': 1, 'bounds': 1}
{'node': 21, 'osm': 1, 'bounds': 1}
{'node': 22, 'osm': 1, 'bounds': 1}
{'node': 23, 'osm': 1, 'bounds': 1}
{'node': 24, 'osm': 1, 'bounds': 1}
{'node': 25, 'osm': 1, 'bounds': 1}
{'node': 26, 'osm': 1, 'bounds': 1}
{'node': 27, 'osm': 1, 'bounds': 1}
{'

In [6]:

import xml.etree.cElementTree as ET
from collections import defaultdict
import re

osm_file = open("dubai_abu-dhabi.osm", "r")

street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
street_types = defaultdict(int)

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()

        street_types[street_type] += 1

def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print "%s: %d" % (k, v) 

def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")

def audit():
    for event, elem in ET.iterparse(osm_file):
        if is_street_name(elem):
            audit_street_type(street_types, elem.attrib['v'])    
    print_sorted_dict(street_types)    


audit()


IndexError: pop from empty stack

In [8]:
import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "dubai_abu-dhabi.osm"  # Replace this with your osm file
SAMPLE_FILE = "dubai_abu-dhabi.osm"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

ParseError: no element found: line 1, column 0